In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools.render import render_text_description
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf

In [3]:
load_dotenv(find_dotenv("../.env"))

True

In [4]:
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
# llm=ChatOpenAI(model="gpt-3.5-turbo")

In [18]:
prompt=ChatPromptTemplate.from_template(
    template="Tell me a 200 words story on {topic}"
)

In [19]:
chain=prompt|llm

<h2> Input Schema </h2>

<h5>The Input Schema of the chain is the input schema of its first part, the prompt</h5>

In [20]:
chain.input_schema.model_json_schema()

{'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic'],
 'title': 'PromptInput',
 'type': 'object'}

In [21]:
prompt.input_schema.model_json_schema()

{'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic'],
 'title': 'PromptInput',
 'type': 'object'}

In [22]:
# Not the same as the the chain and the prompt
llm.input_schema.model_json_schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'},
         {'additionalProperties': True, 'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'additionalProperties': True,
     'title': 'Additional Kwargs',
     'type': 'object'},
    'response_metadata': {'additionalProperties': True,
     'title': 'Response Metadata',
     'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default'

<h3>Output Schema</h3>

<h5>The Output Schema of the chain is the output schema of its last part, in this case a ChatModel, which outputs a ChatMessage</h5>

In [23]:
chain.output_schema.model_json_schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'},
         {'additionalProperties': True, 'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'additionalProperties': True,
     'title': 'Additional Kwargs',
     'type': 'object'},
    'response_metadata': {'additionalProperties': True,
     'title': 'Response Metadata',
     'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default'

In [24]:
llm.output_schema.model_json_schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'},
         {'additionalProperties': True, 'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'additionalProperties': True,
     'title': 'Additional Kwargs',
     'type': 'object'},
    'response_metadata': {'additionalProperties': True,
     'title': 'Response Metadata',
     'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default'

<h3>Stream</h3>

In [26]:
for s in chain.stream(input={'topic':'bear'},):
    print(s.content,end="",flush=True)

The old bear, Barnaby, lumbered through the autumn woods, his thick fur a patchwork of brown and grey. The air was crisp, carrying the scent of decaying leaves and ripe berries. He was searching for one last feast before the long sleep of winter.

He sniffed the air, his nostrils twitching. A faint, sweet aroma drew him towards a clearing. There, nestled amongst the ferns, was a patch of wild raspberries, bursting with juicy redness. Barnaby shuffled forward, his eyes gleaming.

He devoured the berries, their sweetness a burst of summer in the cooling air. He ate until his belly was round and full, a contented rumble emanating from his chest. As the sun dipped below the trees, casting long shadows, Barnaby knew it was time.

He ambled towards his den, a cozy burrow beneath the roots of an ancient oak. He curled up amongst the dried leaves, the taste of raspberries lingering on his tongue. As he drifted off to sleep, he dreamt of spring, of sunshine and blooming flowers, and the promise

<h3>Invoke</h3>

In [27]:
chain.invoke(input={"topic":"bear"}).content

'Barnaby the bear was, by all accounts, a clumsy creature. While his brethren expertly fished salmon from the rushing river, Barnaby tripped over his own paws and ended up face-first in the mud. He yearned to be graceful, to effortlessly pluck berries from the highest branches, but his attempts always ended in comical tumbles.\n\nOne crisp autumn day, the forest echoed with the frantic flapping of wings. A flock of geese, usually a blur in the sky, were struggling against a fierce wind, their formation broken. One small gosling, separated from its mother, spiraled downwards, destined for a rocky landing.\n\nWithout thinking, Barnaby lumbered forward. His clumsy gait transformed into a surprisingly swift run. He positioned himself under the falling gosling, his bulky frame cushioning its descent. The little bird landed gently on his back, unharmed.\n\nThe mother goose, witnessing the rescue, honked her gratitude. Barnaby, usually embarrassed by his clumsiness, felt a warmth bloom in his

<h3>Batch</h3>

In [28]:
chain.batch(inputs=[
    {"topic":"bear"},
    {"topic":"Virat Kohli"}
])

[AIMessage(content="Barnaby the bear was a creature of habit. Every morning, he'd lumber down to the creek, dip his paw in, and fish for salmon. He wasn't the most skilled fisherman, often coming up empty-pawed, but he enjoyed the ritual.\n\nOne morning, however, the creek was unusually quiet. No splashing salmon, no birds chirping, just an eerie stillness. Barnaby sniffed the air, a strange metallic scent stinging his nostrils. He followed the scent upstream and gasped. A section of the forest had been felled, leaving a gaping wound in the landscape. A machine, belching black smoke, sat idle by the creek, polluting the water.\n\nBarnaby felt a surge of anger, a protective instinct roaring within him. This wasn't right. This wasn't his creek anymore. He roared, a deep, guttural sound that echoed through the devastated landscape. The sound was a promise, a warning. Barnaby, the bear of habit, was about to break one. He was about to fight for his home.", additional_kwargs={}, response_me

In [29]:
# Setting Concurrency

chain.batch(
    inputs=[
    {"topic":"bear"},
    {"topic":"Virat Kohli"},
    ],
    config={
        "max_concurrency":5
    }
)

[AIMessage(content='Barnaby the bear loved berries, especially the plump, juicy blueberries that grew on Blueberry Hill. Every morning, he’d lumber up the hill, his nose twitching with anticipation. One day, he arrived to find the bushes stripped bare. A tiny, tearful voice whimpered nearby.\n\nIt was a little field mouse, Millie. “The wind blew all the berries away!” she sobbed, pointing to a scattered pile at the bottom of the hill, too far for her to reach. Barnaby, his own disappointment momentarily forgotten, lumbered down the hill. He carefully gathered the berries in his paws, more gently than one would expect from such a large creature.\n\nHe returned to Millie, presenting her with a mountain of blueberries. Her eyes widened. "For me?" she squeaked. Barnaby nodded, a soft rumble escaping his throat. Millie nibbled a berry, her tears replaced by a happy smile. Barnaby watched, a warmth spreading through him. He realized that sharing the joy of blueberries was even sweeter than e

<h3>Async Stream</h3>

In [31]:
async for s in chain.astream(input={'topic':'bear'}):
    print(s.content,end="",flush=True)

Barnaby the bear lived a simple life. He woke with the sun, fished in the cool river, and napped under the shade of the ancient oak. He wasn't a bear for grand adventures, preferring the quiet comfort of his familiar forest. One crisp autumn day, a strange scent filled the air – sweet and intoxicating. Curiosity, a rare guest in Barnaby's mind, tugged at him.

He followed the aroma, pushing through thorny bushes and scrambling over fallen logs. The scent led him to a clearing, where a human family was picnicking. A basket overflowed with berries, pies, and the source of the irresistible smell: honey cakes.

Barnaby, usually wary of humans, found himself mesmerized. He cautiously approached, his large paws silent on the fallen leaves. The family noticed him, their eyes wide with a mixture of fear and awe. He stopped, unsure what to do. Then, a little girl offered him a honey cake. Barnaby, forgetting all his caution, gently took it, the sweet taste exploding on his tongue. For a moment,

<h3> Async Invoke </h3>

In [32]:
await chain.ainvoke(input={"topic":"Rohit Sharma"})

AIMessage(content="Rohit Sharma, a name synonymous with effortless elegance and destructive power in cricket, wasn't always a superstar. He started as a talented but inconsistent batsman, flashes of brilliance overshadowed by periods of frustrating mediocrity. Doubts swirled around his potential, whispers suggesting he might never truly unlock it.\n\nThen came the breakthrough. Promoted to open the innings for India in ODIs, Rohit found his groove. The extra time to build his innings allowed his natural timing to shine. He began converting starts into big scores, then into double centuries, redefining the limits of what was possible in the format.\n\nHis bat became a conductor, orchestrating symphonies of sixes and fours. He danced down the wicket, flicked deliveries over fine leg, and nonchalantly dispatched even the best bowlers to the boundary. Rohit Sharma, the Hitman, had arrived.\n\nHe went on to captain India, leading with calmness and tactical acumen. He proved that talent, cou

<h3>Async Batch </h3>

In [33]:
await chain.abatch(inputs=[{"topic":"Sachin Tendulkar"},{"topic":"Shane Warne"}])

[AIMessage(content='The roar of the Wankhede Stadium was deafening. Sachin Tendulkar, the "God of Cricket," stood at the crease, his bat raised, acknowledging the adulation. It was his final Test match, a moment etched in the hearts of a billion Indians.\n\nFrom a young age, Sachin\'s passion for cricket was undeniable. He honed his skills relentlessly, his dedication unwavering. His debut at 16 against Pakistan was a baptism by fire, but he emerged, a prodigy ready to conquer the world.\n\nOver two decades, Sachin captivated audiences with his artistry. He was a master of timing, a sculptor of strokes, weaving magic with willow. He battled injuries, faced relentless pressure, and carried the hopes of a nation on his shoulders. He scored a hundred centuries, a feat unmatched and perhaps unmatchable.\n\nThat final innings was a culmination of a legendary career. Every run was celebrated, every boundary cheered. When he finally departed, the stadium erupted in a standing ovation, tears s

<h3>Event Reference</h3>

In [34]:
template="""
Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt=ChatPromptTemplate.from_template(template=template)

vectorStore=FAISS.from_texts(
    texts=["Ritish has a 10 month old son !",
           "Rohit is Ritish's Younger Brother",
           "Eklavya is Ritish's Son"
           "Rekha is the name of Ritish's Mother",
           "Anirban is Ritish's Father"],
    embedding=SentenceTransformerEmbeddings())

retriever=vectorStore.as_retriever()

outputParser=StrOutputParser()

C:\Users\MSI\AppData\Local\Temp\ipykernel_16320\1583948233.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding=SentenceTransformerEmbeddings())
C:\Users\MSI\AppData\Local\Temp\ipykernel_16320\1583948233.py:16: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=SentenceTransformerEmbeddings())
c:\Users\MSI\anaconda3\envs\lcnenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.read

In [35]:
retrievalChain=RunnablePassthrough.assign(context=itemgetter("question")|retriever.with_config(run_name="Docs") ) | prompt | llm.with_config(run_name="my_llm") | outputParser

In [36]:
question={"question":"Who is Anirban's Wife and who is Eklavya's Uncle?"}

In [37]:
print(retrievalChain.invoke(input=question))

Based on the provided information:

*   Anirban's Wife: We don't know the name of Anirban's wife. We only know that Anirban is Ritish's father.
*   Eklavya's Uncle: Rohit is Eklavya's uncle. We know that Eklavya is Ritish's son and Rohit is Ritish's younger brother.


In [38]:
async for event in retrievalChain.astream_events(input=question,
                                                 version="v1",
                                                 include_names=["Docs","my_llm"]
                                                ):
    print(event,"\n")

{'event': 'on_retriever_start', 'name': 'Docs', 'run_id': 'd0104a74-98fd-4b47-9a84-cce4d6b0c8a3', 'tags': ['seq:step:2', 'FAISS', 'HuggingFaceEmbeddings'], 'metadata': {'ls_retriever_name': 'vectorstore', 'ls_vector_store_provider': 'FAISS', 'ls_embedding_provider': 'HuggingFaceEmbeddings'}, 'data': {'input': {'query': "Who is Anirban's Wife and who is Eklavya's Uncle?"}}, 'parent_ids': []} 

{'event': 'on_retriever_end', 'name': 'Docs', 'run_id': 'd0104a74-98fd-4b47-9a84-cce4d6b0c8a3', 'tags': ['seq:step:2', 'FAISS', 'HuggingFaceEmbeddings'], 'metadata': {'ls_retriever_name': 'vectorstore', 'ls_vector_store_provider': 'FAISS', 'ls_embedding_provider': 'HuggingFaceEmbeddings'}, 'data': {'input': {'query': "Who is Anirban's Wife and who is Eklavya's Uncle?"}, 'output': {'documents': [Document(id='ff006209-317b-444c-a9fc-53a071e0d30b', metadata={}, page_content="Anirban is Ritish's Father"), Document(id='d9d3ca99-6542-41fa-8425-5e04603e4456', metadata={}, page_content="Eklavya is Ritish'

In [ ]:
async for event in retrievalChain.astream_events(input=question,
                                                 version="v1",
                                                 include_names=["Docs","my_llm"]
                                                ):
    kind=event['event']
    if kind=="on_chat_model_stream":
        print(event["data"]["chunk"].content,end="|")
    elif kind=="on_chat_model_start":
        print("\nStreaming LLM")
    elif kind=="on_chat_model_end":
        print("Done Streaming LLM.")
    elif kind=="on_retriever_end":
        print("Retrieved the following Documents:")
        print(event["data"]["output"]["documents"])
    elif kind=="on_tool_end":
        print(f"Ended Tool: {event['name']}")

Retrieved the following Documents:
[Document(id='ff006209-317b-444c-a9fc-53a071e0d30b', metadata={}, page_content="Anirban is Ritish's Father"), Document(id='d9d3ca99-6542-41fa-8425-5e04603e4456', metadata={}, page_content="Eklavya is Ritish's SonRekha is the name of Ritish's Mother"), Document(id='4f946bb7-1c22-4e8d-a92e-6770170c32c5', metadata={}, page_content="Rohit is Ritish's Younger Brother"), Document(id='52a32645-9b15-4a84-b7e0-24a5cf925726', metadata={}, page_content='Ritish has a 10 month old son !')]

Streaming LLM
Based| on the provided| information:

*   Anirban's Wife: We don't know who| Anirban's wife is. Anirban is Ritish's Father|, and Rekha is Ritish's Mother, therefore Rekha is Anirban's Wife.
*   Eklavya's Uncle|: Rohit is Ritish's Younger Brother, and Eklavya is Ritish's Son, therefore Rohit is Eklavya's Uncle|.|Done Streaming LLM.


In [43]:
async for event in retrievalChain.astream_events(input=question,
                                                 version="v2",
                                                 include_names=["Docs","my_llm"]
                                                ):
    kind=event['event']
    if kind=="on_chat_model_stream":
        print(event["data"]["chunk"].content,end="|")
    elif kind=="on_chat_model_start":
        print("\nStreaming LLM")
    elif kind=="on_chat_model_end":
        print("Done Streaming LLM.")
    elif kind=="on_retriever_end":
        print("Retrieved the following Documents:")
        print(event['data']['output'])
        # print(event["data"]["output"]["documents"])
    elif kind=="on_tool_end":
        print(f"Ended Tool: {event['name']}")

Retrieved the following Documents:
[Document(id='ff006209-317b-444c-a9fc-53a071e0d30b', metadata={}, page_content="Anirban is Ritish's Father"), Document(id='d9d3ca99-6542-41fa-8425-5e04603e4456', metadata={}, page_content="Eklavya is Ritish's SonRekha is the name of Ritish's Mother"), Document(id='4f946bb7-1c22-4e8d-a92e-6770170c32c5', metadata={}, page_content="Rohit is Ritish's Younger Brother"), Document(id='52a32645-9b15-4a84-b7e0-24a5cf925726', metadata={}, page_content='Ritish has a 10 month old son !')]

Streaming LLM
Based| on the provided information:

*   Anirban's Wife: The document does not| mention who Anirban's wife is.
*   Eklavya|'s Uncle: Rohit is Ritish's younger brother. Eklavya is Ritish's son. Therefore, Rohit is Eklavya's| uncle.|Done Streaming LLM.


In [44]:
async for chunk in retrievalChain.astream_log(input=question,include_names=["Docs"]):
    print("-"*40)
    print(chunk)

----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '51cd7d04-383e-45db-b4a0-29d319fa3f20',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': '1b6c6dff-cadb-4a15-9366-2e8ebddc44d4',
            'metadata': {'ls_embedding_provider': 'HuggingFaceEmbeddings',
                         'ls_retriever_name': 'vectorstore',
                         'ls_vector_store_provider': 'FAISS'},
            'name': 'Docs',
            'start_time': '2025-06-04T13:31:28.709+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:2', 'FAISS', 'HuggingFaceEmbeddings'],
            'type': 'retriever'}})
---------------

In [71]:
## Stream Incremental RunState with diff=False
async for chunk in retrievalChain.astream_log(input=question,include_names=["Docs"],diff=False):
    print("-"*40)
    print(chunk)

----------------------------------------
RunLog({'final_output': None,
 'id': '285c7584-ffc6-4fda-8de4-3a2a96530f6c',
 'logs': {},
 'name': 'RunnableSequence',
 'streamed_output': [],
 'type': 'chain'})
----------------------------------------
RunLog({'final_output': None,
 'id': '285c7584-ffc6-4fda-8de4-3a2a96530f6c',
 'logs': {'Docs': {'end_time': None,
                   'final_output': None,
                   'id': 'd6cba482-cd44-4f72-bab0-7fe738b5d482',
                   'metadata': {'ls_embedding_provider': 'HuggingFaceEmbeddings',
                                'ls_retriever_name': 'vectorstore',
                                'ls_vector_store_provider': 'FAISS'},
                   'name': 'Docs',
                   'start_time': '2024-09-24T20:44:13.157+00:00',
                   'streamed_output': [],
                   'streamed_output_str': [],
                   'tags': ['seq:step:2', 'FAISS', 'HuggingFaceEmbeddings'],
                   'type': 'retriever'}},
 'name':

<h3>Runnable Parallel</h3>

In [48]:
chain1=ChatPromptTemplate.from_template(template="Tell Me a Joke about {topic}")|llm
chain2=ChatPromptTemplate.from_template(template="Write a Poem on the Following {subject}")|llm

In [49]:
combinedChain=RunnableParallel(joke=chain1,poem=chain2)

In [50]:
start=time.time()
print(chain1.invoke(input={"topic":"Titanic"}).content)
print(f"Took {time.time()-start} Seconds")

Why did the iceberg only get one star on Yelp?

Because it had terrible reviews. People said it was a bit too *impactful*.
Took 2.480455160140991 Seconds


In [51]:
start=time.time()
print(chain2.invoke(input={"subject":"Titanic"}).content)
print(f"Took {time.time()-start} Seconds")

In Belfast's heart, a dream took hold,
A ship of dreams, in steel controlled.
Titanic, grand, a floating grace,
A titan born to conquer space.

With maiden voyage, spirits soared,
A floating palace, richly stored.
From Southampton's quay, she bravely sailed,
Where hopes and futures were unveiled.

The decks alive with laughter bright,
Beneath the stars, a dazzling sight.
First class elegance, a world apart,
While steerage dreams held close to heart.

But fate, it lurked, a silent threat,
A frozen mountain, coldly set.
The iceberg loomed, a ghostly white,
A tragic end to darkest night.

A scrape, a shudder, fear took hold,
As icy water, stories told.
The unsinkable, began to weep,
As secrets slumbered in the deep.

The screams, the chaos, filled the air,
As lifeboats lowered, in despair.
A sacrifice, a hero's stand,
As souls departed, from the land.

The ocean claimed her, cold and vast,
A legend born, that forever cast
A shadow on the rolling wave,
A watery tomb, a tragic grave.

Thoug

In [52]:
start=time.time()
print(combinedChain.invoke(input={"subject":"Titanic","topic":"Titanic"}))
print(f"Took {time.time()-start} Seconds")

{'joke': AIMessage(content='Why did the iceberg get a bad reputation?\n\nBecause it sank a lot of relationships.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-8c045317-3103-4ee4-8f4d-48fe5368d52a-0', usage_metadata={'input_tokens': 6, 'output_tokens': 19, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}}), 'poem': AIMessage(content="The Dream of Iron, forged in fire's heart,\nA floating palace, set to play its part.\nTitanic, queen of ocean, bold and grand,\nA marvel built by human skill and hand.\n\nShe slipped her moorings, Belfast left behind,\nA promise whispered on the salty wind.\nOf gilded halls and dinners, rich and deep,\nWhere fortunes mingled, secrets they would keep.\n\nThe steerage huddled, hopes within their breast,\nA new life beckoned, westward to the West.\nThey dreamt of freedom, land and open sky,\nUnburd

In [53]:
RunnableParallel({'joke':chain1,'poem':chain2}).invoke(input={"subject":"Titanic","topic":"Titanic"})


{'joke': AIMessage(content="Why couldn't the Titanic play poker?\n\nBecause it was always going under the table!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-e627d964-87c9-4a02-9a2f-2ca6696583fa-0', usage_metadata={'input_tokens': 6, 'output_tokens': 20, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}}),
 'poem': AIMessage(content="The ocean's breath, a chilling sigh,\nAs she, a dream, began to fly.\nTitanic, queen of sea and steel,\nA floating palace, meant to heal\nThe distance 'twixt the old and new,\nA marvel born for me and you.\n\nHer decks alive with vibrant cheer,\nFirst class dreams, and second fear\nOf futures unknown, hopes held tight,\nAs steerage passengers found light\nIn promises of lands afar,\nBeneath a hopeful, guiding star.\n\nThe engines hummed a steady beat,\nA rhythm strong, a pulsing heat.\nThe stok

<h3> Parallelism on Batches</h3>

In [54]:
start=time.time()
print(chain1.batch(inputs=[{"topic":"Titanic"},{"topic":"Kolkata"}]))
print(f"Took {time.time()-start} Seconds")

[AIMessage(content="Why couldn't the Titanic play poker?\n\nBecause it kept going under the table!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-64ca8192-6242-46ad-b491-565fd221da1f-0', usage_metadata={'input_tokens': 6, 'output_tokens': 19, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}}), AIMessage(content="Why did the rasgulla cross the road in Kolkata?\n\nTo prove it wasn't afraid of a little *mishti* business!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-e56d59ab-8be7-4ba3-8c8f-303e9dec8924-0', usage_metadata={'input_tokens': 6, 'output_tokens': 30, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}})]
Took 2.087763786315918 Seconds


In [55]:
start=time.time()
print(chain2.batch(inputs=[{"subject":"Titanic"},{"subject":"Kolkata"}]))
print(f"Took {time.time()-start} Seconds")

[AIMessage(content='The pride of the White Star Line, a dream in steel,\nTitanic, a floating palace, destined to appeal.\nA marvel of engineering, a testament to might,\nSailing forth on a starlit, fateful night.\n\nThe finest silks and silver, a symphony of grace,\nA microcosm of society, in this luxurious space.\nFrom opulent suites to steerage\'s crowded hold,\nA tapestry of lives, a story to unfold.\n\nThe engines hummed a lullaby, a rhythmic, steady beat,\nUnheeding of the dangers, the icy, frigid threat.\nThen, a shadow in the darkness, a berg of frozen pain,\nA whisper of disaster, a chilling, fatal strain.\n\nThe shudder ran through the vessel, a tremor, sharp and deep,\nAwakening the slumbering souls from their peaceful sleep.\nA gash below the waterline, a silent, deadly wound,\nSealing the fate of many, on that watery ground.\n\n"Women and children first," the order, bravely cried,\nAs chaos and confusion began to coincide.\nLifeboats lowered slowly, into the icy black,\nLea

In [56]:
start=time.time()
print(combinedChain.batch(inputs=[
                                    {"subject":"Titanic","topic":"Titanic"},  # Joke on Titanic and Poem on Titanic
                                    {"subject":"Kolkata","topic":"Kolkata"}  # Joke on Kolkata and Poem on Kolkata
                          ]))
print(f"Took {time.time()-start} Seconds")

[{'joke': AIMessage(content='Why did the iceberg only get one star on Yelp?\n\nBecause it ruined a cruise and gave everyone chills!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-ca5f88a9-f31c-48b2-8912-422433ec3137-0', usage_metadata={'input_tokens': 6, 'output_tokens': 23, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}}), 'poem': AIMessage(content="The iron lung, a titan's dream,\nA floating palace, light agleam.\nFrom Southampton's shore, she bravely sailed,\nA world of wonder, stories veiled.\n\nFirst class shimmered, crystal bright,\nWith silks and satins, bathed in light.\nWhile steerage huddled, hopes held dear,\nFor new beginnings, banishing fear.\n\nThe ocean stretched, a canvas vast,\nBeneath the stars, the die was cast.\nA fateful night, a chilling breath,\nAn icy giant, courting death.\n\nA scrape, a shudder, s

In [57]:
combinedChain.get_graph().print_ascii()

                       +--------------------------+                        
                       | Parallel<joke,poem>Input |                        
                       +--------------------------+                        
                       *****                   *****                       
                    ***                             ***                    
                 ***                                   ***                 
  +--------------------+                          +--------------------+   
  | ChatPromptTemplate |                          | ChatPromptTemplate |   
  +--------------------+                          +--------------------+   
             *                                               *             
             *                                               *             
             *                                               *             
+------------------------+                      +------------------------+ 
| ChatGoogle

In [58]:
combinedChain.get_prompts()

[ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell Me a Joke about {topic}'), additional_kwargs={})]),
 ChatPromptTemplate(input_variables=['subject'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['subject'], input_types={}, partial_variables={}, template='Write a Poem on the Following {subject}'), additional_kwargs={})])]